In [1]:
import pandas as pd

In [2]:
excel_file = pd.ExcelFile("singleplex_reaction_data.xlsx")
sheet_names = excel_file.sheet_names
seq_dfs = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

print(f"Loaded sheets: {list(seq_dfs.keys())}")

Loaded sheets: ['reaction_summaries', 'amplicons', 'cores', 'fragments', 'primers', 'probes', 'readings']


In [3]:
excel_file = pd.ExcelFile("competitive_reaction_data.xlsx")
sheet_names = excel_file.sheet_names
fig_dfs = {sheet_name: excel_file.parse(sheet_name) for sheet_name in sheet_names}

print(f"Loaded sheets: {list(fig_dfs.keys())}")

Loaded sheets: ['reaction_data', 'reaction_metadata', 'reaction_figures', 'supermix_components', 'fragment_sequences', 'primer_sequences', 'probe_sequences']


In [4]:
fragment_rates = (
    seq_dfs["reaction_summaries"][["amplicon_id", "rate"]]
    .groupby("amplicon_id")
    .mean()
    .reset_index()
    .sort_values("rate", ascending=False)
    .merge(seq_dfs["amplicons"][["amplicon_id", "name"]])
    .rename(columns={"name": "amplicon"})
    .merge(
        seq_dfs["fragments"][["fragment_id", "name", "amplicon_id"]].rename(
            columns={"name": "fragment"}
        )
    )[["fragment", "rate"]]
    .set_index("fragment")
    .rate
)


In [27]:
def retrieve_fragment_rates(figure):
    these_rxns = fig_dfs["reaction_figures"][
        fig_dfs["reaction_figures"].figure.str.startswith(figure)
    ]
    rxn_fragments = (
        fig_dfs["reaction_metadata"][["Reaction", "component"]]
        .drop_duplicates()
        .query("component.str.contains('S')")
        .rename(columns={"component": "Target"})
    )
    mix_fragments = (
        fig_dfs["supermix_components"]
        .query("component.str.contains('S')")[["SuperMix", "component"]]
        .rename(columns={"component": "Competitor"})
    )

    figure_fragments = (
        these_rxns.merge(rxn_fragments, on="Reaction", how="left")
        .drop_duplicates(["supermix", "figure", "Target"])
        .merge(mix_fragments, left_on="supermix", right_on="SuperMix", how="left")[
            ["figure", "Target", "Competitor"]
        ]
        .assign(
            Target_rate=lambda df: df.Target.str.removesuffix('b').map(fragment_rates),
            Competitor_rate=lambda df: df.Competitor.str.removesuffix('b').map(fragment_rates),
        )
    )

    return figure_fragments

In [28]:
retrieve_fragment_rates("S2B")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,S2B_i,S056.4.12,S056.4.3,0.797897,0.476629
1,S2B_ii,S056.4.12,S056.4.1,0.797897,0.834956
2,S2B_iii,S044.13,S044.14,0.863789,0.746113
3,S2B_iv,S056.4.12,S056.4.4,0.797897,0.735695
4,S2B_v,S044.13,S044.9,0.863789,0.740611


In [29]:
retrieve_fragment_rates("S3A")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,S3A,S036.5,NaN,0.821959,NaN
1,S3A,S057.3.2,NaN,0.518897,NaN
2,S3A,S057.4.2,NaN,0.497023,NaN


In [30]:
retrieve_fragment_rates("S4A")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,S4A,S036.0,NaN,0.708326,NaN
1,S4A,S057.0.1,NaN,0.643763,NaN
2,S4A,S057.1.2,NaN,0.650153,NaN


In [31]:
retrieve_fragment_rates("S5A")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,S5A,S037.01.01b,NaN,0.636401,NaN
1,S5A,S036.1b,NaN,0.711474,NaN


In [32]:
retrieve_fragment_rates("S5C")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,S5C,S037.01.01b,NaN,0.636401,NaN
1,S5C,S036.0,NaN,0.708326,NaN


In [35]:
from candas.lims import Librarian, library
from candas.lims.schema import Fragment, Amplicon, Core, Primer, ProbeOligo, ProbeSeq

lib = Librarian().load(library).open()

In [42]:
for name in ['S036.5', 'S057.3.2', 'S057.4.2']:

    f = lib.Session().query(
        Fragment
    ).filter(Fragment.name == name).one()
    
    a = f.amplicon
    primers = a.primers
    probe = a.core.probe
    
    print(f.name, [p.name for p in primers], probe.oligos[0].f)

S036.5 ['FP001', 'RP001'] HEX
S057.3.2 ['FP001', 'RP005'] FAM
S057.4.2 ['FP005', 'RP005'] HEX


In [33]:
retrieve_fragment_rates("3B")

,figure,Target,Competitor,Target_rate,Competitor_rate
0,3Bi,S036.5,S057.3.2,0.821959,0.518897
1,3Bi,S036.5,S057.4.2,0.821959,0.497023
2,3Bi,S057.3.2,S057.3.2,0.518897,0.518897
3,3Bi,S057.3.2,S057.4.2,0.518897,0.497023
4,3Bi,S057.4.2,S057.3.2,0.497023,0.518897
5,3Bi,S057.4.2,S057.4.2,0.497023,0.497023
6,3Bii,S036.5,S057.3.2,0.821959,0.518897
7,3Bii,S036.5,S057.4.2,0.821959,0.497023
8,3Bii,S057.3.2,S057.3.2,0.518897,0.518897
9,3Bii,S057.3.2,S057.4.2,0.518897,0.497023
